In [51]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy import *
from sklearn.preprocessing import LabelEncoder
import gzip
import warnings
warnings.filterwarnings('ignore')

In [52]:
# data = pd.read_json(r'/Users/lomesh/Downloads/amazon_dataset/reviews_Books_5.json')

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/Users/lomesh/Downloads/reviews_Clothing_Shoes_and_Jewelry_5.json.gz')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 278677 rows and 9 columns


In [53]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014"
...,...,...,...,...,...,...,...,...,...
278672,A2DG63DN704LOI,B00KKXCJQU,ESlayd,"[0, 1]",I don't normally go ga-ga over a product very ...,5.0,These have literally changed the way I travel!,1403568000,"06 24, 2014"
278673,A1EVV74UQYVKRY,B00KKXCJQU,K. Groh,"[0, 0]",I've been traveling back and forth to England ...,5.0,Wonderful for Better Packing,1403740800,"06 26, 2014"
278674,A1UQBFCERIP7VJ,B00KKXCJQU,Margaret Picky,"[1, 1]",These are very nice packing cubes and the 18 x...,5.0,"Convenient, lightweight, and durable",1402185600,"06 8, 2014"
278675,A22CW0ZHY3NJH8,B00KKXCJQU,Noname,"[2, 2]",I am on vacation with my family of four and th...,5.0,Holds Up Well In Real World Test,1404691200,"07 7, 2014"


In [54]:
data = df[['reviewerID', 'asin', 'overall']]

In [55]:
labelencoder = LabelEncoder()
data['reviewerID'] = labelencoder.fit_transform(data['reviewerID'])
data['asin'] = labelencoder.fit_transform(data['asin'])

In [56]:
data

,reviewerID,asin,overall
0,5862,0,5.0
1,15118,0,5.0
2,7923,0,5.0
3,31651,0,5.0
4,25705,0,5.0
...,...,...,...
278672,14317,23032,5.0
278673,4204,23032,5.0
278674,8843,23032,5.0
278675,11096,23032,5.0


In [57]:
users = np.array(data.iloc[:,0])
items = np.array(data.iloc[:,1])
ratings = np.array(data.iloc[:,2])

In [58]:
len(users), len(items), len(ratings)

(278677, 278677, 278677)

In [59]:
utility_matrix = csr_matrix((ratings, (users, items))).toarray() # Users x Items 
utility_matrix_t = utility_matrix.T
utility_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
print(utility_matrix.shape)
print(utility_matrix_t.shape)

(39387, 23033)
(23033, 39387)


In [62]:
np.savetxt('/Users/lomesh/Downloads/users.txt', utility_matrix, fmt="%d") # All rows as a USERS
np.savetxt('/Users/lomesh/Downloads/items.txt', utility_matrix_t, fmt="%d") # All rows as a ITEMS